In [1]:
import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import models.loadImg as loadImg
import models.modelsCNN as modelsCNN
import models.optimizer as optimizer

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import scipy.stats
from sklearn.metrics import pairwise_distances

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3'
use_cuda=True
datadir='/media/xinyi/dcis2idc/data'
name='exp0'
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name
sampledir=plotsavepath
clustersavedir_alltrain=os.path.join(sampledir,'cluster_alltrain_reordered')
ep=311
with open(os.path.join(datadir,'processed','train_cnnvae_names'), 'rb') as input:
    allImgNames=pickle.load(input)
#plot by disease progression
br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']

progList=np.copy(allImgNames)
for s in np.unique(allImgNames):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList[allImgNames==s]=prog_s
    




In [3]:
savenamesample='alltrain'

neworder=[1, 5, 3, 7, 2, 0, 4, 6]
#use chosen subcluster number and save plots
scanpy.settings.verbosity = 3
# subcluster=8
subclusterDict={0:[4],1:[6],2:[8],3:[6],4:[6],5:[6],6:[6],7:[4]}
ncluster=8

plotepoch=311
clusterplotdir=os.path.join(clustersavedir_alltrain,'plots')
n_pcs=50
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
with open(os.path.join(clustersavedir_alltrain,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
    clusterRes=pickle.load(output)

kmeans_sub=(np.zeros(clusterRes.size)-1).astype(str)
savenameAdd='_plottingIdx_progBalanced_'+str(0)
subclusternumbers=[4,6,8,6,6,6,6,4]
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+savenameAdd
for cnew in np.unique(clusterRes):
#     print('cluster'+str(c))
    c=neworder[cnew]
    
    subclustersavedir_alltrain=os.path.join(clustersavedir_alltrain,savenamecluster+'_subcluster'+str(c))
    with open(os.path.join(subclustersavedir_alltrain,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes=pickle.load(output)
    print(np.unique(subclusterRes))
    kmeans_sub[clusterRes==cnew]=np.char.add(np.repeat(str(cnew)+'-',subclusterRes.size),subclusterRes.astype(str))
   

[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3]
[0 1 2 3 4 5 6 7]
[0 1 2 3]
[0 1 2 3 4 5]
[0 1 2 3 4 5]


In [4]:
with open(os.path.join(datadir,'processed','train_cnnvae_coord'), 'rb') as output:
    coordlist=pickle.load(output)

In [5]:
for p in np.unique(progList):
    if p=='Ductal carcinoma in situ':
        progList[progList==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ and breast tissue':
        progList[progList==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ with early infiltratio':
        progList[progList==p]='DCIS with early infiltration'
    
    elif p=='Micropapillary type ductal carcinoma in situ wi':
        progList[progList==p]='DCIS with early infiltration'    
#     elif p=='Atypical hyperlasia':
#         progList[progList==p]='Hyperplasia'

In [6]:
progInclude=np.array(['Breast tissue','Cancer adjacent normal breast tissue','Hyperplasia','DCIS and breast tissue',
                     'DCIS with early infiltration','Invasive ductal carcinoma and breast tissue','Invasive ductal carcinoma'])

In [7]:
progIncludeIdx=np.repeat(False,progList.size)
for p in progInclude:
    progIncludeIdx[progList==p]=True
    
coordlist=coordlist[progIncludeIdx]
allImgNames=allImgNames[progIncludeIdx]
clusterRes=clusterRes[progIncludeIdx]
kmeans_sub=kmeans_sub[progIncludeIdx]
progList=progList[progIncludeIdx]

In [8]:
sUnique,sidx_start=np.unique(allImgNames,return_index=True)
progUnique,labels_train,progCounts=np.unique(progList[sidx_start],return_counts=True,return_inverse=True)
for p in range(progUnique.size):
    print(progUnique[p])
    print(progCounts[p])

Breast tissue
20
Cancer adjacent normal breast tissue
13
DCIS and breast tissue
16
DCIS with early infiltration
30
Hyperplasia
41
Invasive ductal carcinoma
70
Invasive ductal carcinoma and breast tissue
8


In [9]:
#get cluster composition
clusterUnique=np.unique(clusterRes)
subclusterUnique=np.unique(kmeans_sub)
inputCluster=np.zeros((sUnique.size,clusterUnique.size))
inputSubcluster=np.zeros((sUnique.size,subclusterUnique.size))
for i in range(sUnique.size):
    clusterRes_i=clusterRes[allImgNames==sUnique[i]]
    kmeans_sub_i=kmeans_sub[allImgNames==sUnique[i]]
    for j in range(clusterUnique.size):
        inputCluster[i,j]=np.sum(clusterRes_i==clusterUnique[j])
    for j in range(subclusterUnique.size):
        inputSubcluster[i,j]=np.sum(kmeans_sub_i==subclusterUnique[j])
inputCluster=inputCluster/np.sum(inputCluster,axis=1,keepdims=True)
inputSubcluster=inputSubcluster/np.sum(inputSubcluster,axis=1,keepdims=True)

In [10]:
_,inputCounts=np.unique(allImgNames,return_counts=True)
inputAll=np.concatenate((np.concatenate((inputCluster,inputSubcluster),axis=1),inputCounts.reshape(-1,1)),axis=1)

In [11]:
#val cores (as validation cores) and val samples (as test cores)
clustersavedir_valcores=os.path.join(sampledir,'cluster_valcores_reordered')
clustersavedir_valsamples=os.path.join(sampledir,'cluster_valsamples_reordered')

with open(os.path.join(datadir,'processed','train_cnnvae_coord_valcores'), 'rb') as output:
    coordlist_valcores=pickle.load(output)
with open(os.path.join(datadir,'processed','train_cnnvae_coord_valsamples'), 'rb') as output:
    coordlist_valsamples=pickle.load(output)

savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
with open(os.path.join(clustersavedir_valcores,savenamecluster+'_all'), 'rb') as output:
    clusterRes_valcores=pickle.load(output)
with open(os.path.join(clustersavedir_valsamples,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
    clusterRes_valsamples=pickle.load(output)
    
kmeans_sub_valcores=(np.zeros(clusterRes_valcores.size)-1).astype(str)
for c in np.unique(clusterRes_valcores):
    subclustersavedir=os.path.join(clustersavedir_valcores,savenamecluster+'_plottingIdx_progBalanced_'+str(0)+'_subcluster'+str(neworder[c]))
    with open(os.path.join(subclustersavedir,'minibatchkmean_ncluster'+str(subclusterDict[neworder[c]][0])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes=pickle.load(output)
    kmeans_sub_valcores[clusterRes_valcores==c]=np.char.add(np.repeat(str(c)+'-',subclusterRes.size),subclusterRes.astype(str))
    
kmeans_sub_valsamples=(np.zeros(clusterRes_valsamples.size)-1).astype(str)
for c in np.unique(clusterRes_valsamples):
    subclustersavedir=os.path.join(clustersavedir_valsamples,savenamecluster+'_plottingIdx_progBalanced_'+str(0)+'_subcluster'+str(neworder[c]))
    with open(os.path.join(subclustersavedir,'minibatchkmean_ncluster'+str(subclusterDict[neworder[c]][0])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
        subclusterRes=pickle.load(output)
    kmeans_sub_valsamples[clusterRes_valsamples==c]=np.char.add(np.repeat(str(c)+'-',subclusterRes.size),subclusterRes.astype(str))
                

In [12]:
with open(os.path.join(datadir,'processed','train_cnnvae_names_valcores'), 'rb') as input:
    allImgNames_valcores=pickle.load(input)
with open(os.path.join(datadir,'processed','train_cnnvae_names_valsamples'), 'rb') as input:
    allImgNames_valsamples=pickle.load(input)

In [13]:
#plot by disease progression
br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']

progList_valcores=np.copy(allImgNames_valcores)
for s in np.unique(allImgNames_valcores):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList_valcores[allImgNames_valcores==s]=prog_s
    
progList_valsamples=np.copy(allImgNames_valsamples)
for s in np.unique(allImgNames_valsamples):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList_valsamples[allImgNames_valsamples==s]=prog_s
    



In [14]:
np.unique(progList_valsamples)


array(['Atypical hyperplasia', 'Breast tissue',
       'Cancer adjacent normal breast tissue', 'Ductal carcinoma in situ',
       'Ductal carcinoma in situ and breast tissue',
       'Ductal carcinoma in situ with early infiltrati', 'Hyperplasia',
       'Hyperplasia with saccular dilatation',
       'Invasive ductal carcinoma',
       'Invasive ductal carcinoma (breast tissue)',
       'Invasive ductal carcinoma and breast tissue',
       'Micropapillary type ductal carcinoma in situ w'], dtype='<U46')

In [15]:
for p in np.unique(progList_valcores):
    if p=='Ductal carcinoma in situ':
        progList_valcores[progList_valcores==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ and breast tissue':
        progList_valcores[progList_valcores==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ with early infiltratio':
        progList_valcores[progList_valcores==p]='DCIS with early infiltration'
    
    elif p=='Micropapillary type ductal carcinoma in situ wi':
        progList_valcores[progList_valcores==p]='DCIS with early infiltration'    
#     elif p=='Atypical hyperlasia':
#         progList_valcores[progList_valcores==p]='Hyperplasia'

In [16]:
for p in np.unique(progList_valsamples):
    if p=='Ductal carcinoma in situ':
        progList_valsamples[progList_valsamples==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ and breast tissue':
        progList_valsamples[progList_valsamples==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ with early infiltrati':
        progList_valsamples[progList_valsamples==p]='DCIS with early infiltration'
    
    elif p=='Micropapillary type ductal carcinoma in situ w':
        progList_valsamples[progList_valsamples==p]='DCIS with early infiltration'    
#     elif p=='Atypical hyperlasia':
#         progList_valsamples[progList_valsamples==p]='Hyperplasia'

In [17]:
progIncludeIdx_valcores=np.repeat(False,progList_valcores.size)
for p in progInclude:
    progIncludeIdx_valcores[progList_valcores==p]=True
    
progIncludeIdx_valsamples=np.repeat(False,progList_valsamples.size)
for p in progInclude:
    progIncludeIdx_valsamples[progList_valsamples==p]=True
    
coordlist_valcores=coordlist_valcores[progIncludeIdx_valcores]
allImgNames_valcores=allImgNames_valcores[progIncludeIdx_valcores]
clusterRes_valcores=clusterRes_valcores[progIncludeIdx_valcores]
kmeans_sub_valcores=kmeans_sub_valcores[progIncludeIdx_valcores]
progList_valcores=progList_valcores[progIncludeIdx_valcores]

coordlist_valsamples=coordlist_valsamples[progIncludeIdx_valsamples]
allImgNames_valsamples=allImgNames_valsamples[progIncludeIdx_valsamples]
clusterRes_valsamples=clusterRes_valsamples[progIncludeIdx_valsamples]
kmeans_sub_valsamples=kmeans_sub_valsamples[progIncludeIdx_valsamples]
progList_valsamples=progList_valsamples[progIncludeIdx_valsamples]


In [18]:
sUnique_valcores,sidx_start_valcores=np.unique(allImgNames_valcores,return_index=True)
progUnique_valcores,progCounts_valcores=np.unique(progList_valcores[sidx_start_valcores],return_counts=True)
for p in range(progUnique_valcores.size):
    print(progUnique_valcores[p])
    print(progCounts_valcores[p])

Breast tissue
20
Cancer adjacent normal breast tissue
1
Hyperplasia
35
Invasive ductal carcinoma
97


In [19]:
sUnique_valsamples,sidx_start_valsamples=np.unique(allImgNames_valsamples,return_index=True)
progUnique_valsamples,progCounts_valsamples=np.unique(progList_valsamples[sidx_start_valsamples],return_counts=True)
for p in range(progUnique_valsamples.size):
    print(progUnique_valsamples[p])
    print(progCounts_valsamples[p])

Breast tissue
14
Cancer adjacent normal breast tissue
4
DCIS and breast tissue
16
DCIS with early infiltration
29
Hyperplasia
25
Invasive ductal carcinoma
66
Invasive ductal carcinoma and breast tissue
8


In [20]:
#construct labels
labels_valcores=np.zeros(progList_valcores[sidx_start_valcores].size)
for i in range(progUnique.size):
    labels_valcores[progList_valcores[sidx_start_valcores]==progUnique[i]]=i

In [21]:
#construct labels
labels_valsamples=np.zeros(progList_valsamples[sidx_start_valsamples].size)
for i in range(progUnique.size):
    labels_valsamples[progList_valsamples[sidx_start_valsamples]==progUnique[i]]=i

In [22]:
#get cluster composition
inputCluster_valcores=np.zeros((sUnique_valcores.size,clusterUnique.size))
inputSubcluster_valcores=np.zeros((sUnique_valcores.size,subclusterUnique.size))
for i in range(sUnique_valcores.size):
    clusterRes_i=clusterRes_valcores[allImgNames_valcores==sUnique_valcores[i]]
    kmeans_sub_i=kmeans_sub_valcores[allImgNames_valcores==sUnique_valcores[i]]
    for j in range(clusterUnique.size):
        inputCluster_valcores[i,j]=np.sum(clusterRes_i==clusterUnique[j])
    for j in range(subclusterUnique.size):
        inputSubcluster_valcores[i,j]=np.sum(kmeans_sub_i==subclusterUnique[j])
inputCluster_valcores=inputCluster_valcores/np.sum(inputCluster_valcores,axis=1,keepdims=True)
inputSubcluster_valcores=inputSubcluster_valcores/np.sum(inputSubcluster_valcores,axis=1,keepdims=True)

In [23]:
#get cluster composition
inputCluster_valsamples=np.zeros((sUnique_valsamples.size,clusterUnique.size))
inputSubcluster_valsamples=np.zeros((sUnique_valsamples.size,subclusterUnique.size))
for i in range(sUnique_valsamples.size):
    clusterRes_i=clusterRes_valsamples[allImgNames_valsamples==sUnique_valsamples[i]]
    kmeans_sub_i=kmeans_sub_valsamples[allImgNames_valsamples==sUnique_valsamples[i]]
    for j in range(clusterUnique.size):
        inputCluster_valsamples[i,j]=np.sum(clusterRes_i==clusterUnique[j])
    for j in range(subclusterUnique.size):
        inputSubcluster_valsamples[i,j]=np.sum(kmeans_sub_i==subclusterUnique[j])
inputCluster_valsamples=inputCluster_valsamples/np.sum(inputCluster_valsamples,axis=1,keepdims=True)
inputSubcluster_valsamples=inputSubcluster_valsamples/np.sum(inputSubcluster_valsamples,axis=1,keepdims=True)

In [24]:
_,inputCounts_valcores=np.unique(allImgNames_valcores,return_counts=True)
inputAll_valcores=np.concatenate((np.concatenate((inputCluster_valcores,inputSubcluster_valcores),axis=1),inputCounts_valcores.reshape(-1,1)),axis=1)

In [25]:
_,inputCounts_valsamples=np.unique(allImgNames_valsamples,return_counts=True)
inputAll_valsamples=np.concatenate((np.concatenate((inputCluster_valsamples,inputSubcluster_valsamples),axis=1),inputCounts_valsamples.reshape(-1,1)),axis=1)

In [26]:
#normalize count
with open(os.path.join(datadir,'processed','imgSizeByPath'), 'rb') as output:
    imgSize_median=pickle.load(output)
area_train=np.zeros(labels_train.size)
for s in range(labels_train.size):
    area_train[s]=imgSize_median[progUnique[labels_train.astype(int)][s]]
inputAll[:,-1]=inputAll[:,-1]/area_train

In [27]:
area_valcores=np.zeros(labels_valcores.size)
for s in range(labels_valcores.size):
    area_valcores[s]=imgSize_median[progUnique[labels_valcores.astype(int)][s]]
inputAll_valcores[:,-1]=inputAll_valcores[:,-1]/area_valcores

In [28]:
area_valsamples=np.zeros(labels_valsamples.size)
for s in range(labels_valsamples.size):
    area_valsamples[s]=imgSize_median[progUnique[labels_valsamples.astype(int)][s]]
inputAll_valsamples[:,-1]=inputAll_valsamples[:,-1]/area_valsamples

In [29]:
#concatenate cores
inputAll=np.concatenate((inputAll,np.concatenate((inputAll_valcores,inputAll_valsamples),axis=0)),axis=0)
imgNamesAll=np.concatenate((allImgNames[sidx_start],np.concatenate((allImgNames_valcores[sidx_start_valcores],allImgNames_valsamples[sidx_start_valsamples]))))
labelsAll=np.concatenate((labels_train,np.concatenate((labels_valcores,labels_valsamples))))

In [30]:
_,progCountsAll=np.unique(labelsAll,return_counts=True)
weights_train=np.sum(progCountsAll)/progCountsAll

In [31]:
seed=3
epochs=6000
saveFreq=200
lr=0.001 #initial learning rate
weight_decay=0 

# batchsize=4
batchsize=6000
model_str='fc3'

fc_dim1=64
fc_dim2=64
fc_dim3=64


dropout=0.01

name='exp0_pathologyClf_neighbor_clusters_exp0_subset_clusterOnly_countAreaNorm_crossVal'
logsavepath='/media/xinyi/dcis2idc/log/cnnvae'+name
modelsavepath='/media/xinyi/dcis2idc/models/cnnvae'+name
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name


if not os.path.exists(logsavepath):
    os.mkdir(logsavepath)
if not os.path.exists(modelsavepath):
    os.mkdir(modelsavepath)
if not os.path.exists(plotsavepath):
    os.mkdir(plotsavepath)



In [32]:
inputAll=scipy.stats.zscore(inputAll,axis=0)

In [33]:
def train(epoch,trainInput,labels_train):
    t = time.time()
    model.train()
    optimizer.zero_grad()

    pred = model(trainInput)

    loss=lossCE(pred,labels_train)

    loss.backward()
    optimizer.step()

    if epoch%500==0:
        print('Epoch: {:04d}'.format(epoch),
              'loss_train: {:.4f}'.format(loss))
    return loss.item()

In [34]:
inputAll=torch.tensor(inputAll).cuda().float()
labelsAll=torch.tensor(labelsAll).cuda().long()

testepoch=5800
predtest=np.zeros((inputAll.shape[0],np.unique(labels_train).size))
for sampleIdx in range(inputAll.shape[0]):

    trainIdx=np.arange(inputAll.shape[0])!=sampleIdx
    
    seed=3
    torch.manual_seed(seed)
    nclasses=np.unique(labels_train).size
    if use_cuda:
        torch.cuda.manual_seed(seed)

    nfeatures=inputAll.shape[1]
    if model_str=='fc3':
        model = modelsCNN.FC_l3(nfeatures,fc_dim1,fc_dim2,fc_dim3,nclasses,0.5,regrs=False)
        lossCE=torch.nn.CrossEntropyLoss(torch.tensor(weights_train).cuda().float())
    if model_str=='fc5':
        model = modelsCNN.FC_l5(nfeatures,fc_dim1,fc_dim2,fc_dim3,fc_dim4,fc_dim5,nclasses,0.5,regrs=False)
        lossCE=torch.nn.CrossEntropyLoss(torch.tensor(weights_train).cuda().float())
    if model_str=='fc1':
        model = modelsCNN.FC_l1(nfeatures,fc_dim1,nclasses,regrs=False)
        lossCE=torch.nn.CrossEntropyLoss(torch.tensor(weights_train).cuda().float())
    if model_str=='fc0':
        model = modelsCNN.FC_l0(nfeatures,nclasses,regrs=False)
        lossCE=torch.nn.CrossEntropyLoss(torch.tensor(weights_train).cuda().float())

    if use_cuda:
        model.cuda()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_loss_ep=[None]*epochs
    val_loss_ep=[None]*epochs
    t_ep=time.time()

    for ep in range(epochs):
        train_loss_ep[ep]=train(ep,inputAll[trainIdx],labelsAll[trainIdx])


        if ep%saveFreq == 0 and ep!=0:
            torch.save(model.cpu().state_dict(), os.path.join(modelsavepath,imgNamesAll[sampleIdx]+'_'+str(ep)+'.pt'))
        if use_cuda:
            model.cuda()
            torch.cuda.empty_cache()
    print(' total time: {:.4f}s'.format(time.time() - t_ep))

    with open(os.path.join(logsavepath,imgNamesAll[sampleIdx]+'_train_loss'), 'wb') as output:
        pickle.dump(train_loss_ep, output, pickle.HIGHEST_PROTOCOL)

    model.load_state_dict(torch.load(os.path.join(modelsavepath,imgNamesAll[sampleIdx]+'_'+str(testepoch)+'.pt')))
    with torch.no_grad():
        model.cuda()
        model.eval()
        pred = model(inputAll[[sampleIdx]])
        predtest[sampleIdx]=pred.cpu().detach().numpy()

        loss_test=lossCE(pred,labelsAll[[sampleIdx]]).item()

    print(loss_test)

Epoch: 0000 loss_train: 1.9548
Epoch: 0500 loss_train: 0.3760
Epoch: 1000 loss_train: 0.2622
Epoch: 1500 loss_train: 0.1429
Epoch: 2000 loss_train: 0.1070
Epoch: 2500 loss_train: 0.0939
Epoch: 3000 loss_train: 0.0686
Epoch: 3500 loss_train: 0.0889
Epoch: 4000 loss_train: 0.0668
Epoch: 4500 loss_train: 0.0502
Epoch: 5000 loss_train: 0.0661
Epoch: 5500 loss_train: 0.0692
 total time: 11.9004s
6.794906312279636e-06
Epoch: 0000 loss_train: 1.9549
Epoch: 0500 loss_train: 0.3698
Epoch: 1000 loss_train: 0.2632
Epoch: 1500 loss_train: 0.1272
Epoch: 2000 loss_train: 0.0995
Epoch: 2500 loss_train: 0.1194
Epoch: 3000 loss_train: 0.0822
Epoch: 3500 loss_train: 0.0853
Epoch: 4000 loss_train: 0.0637
Epoch: 4500 loss_train: 0.0431
Epoch: 5000 loss_train: 0.0612
Epoch: 5500 loss_train: 0.0711
 total time: 10.5424s
0.0006169561529532075
Epoch: 0000 loss_train: 1.9548
Epoch: 0500 loss_train: 0.3666
Epoch: 1000 loss_train: 0.2777
Epoch: 1500 loss_train: 0.1492
Epoch: 2000 loss_train: 0.1113
Epoch: 2500 l

Epoch: 0500 loss_train: 0.3706
Epoch: 1000 loss_train: 0.2651
Epoch: 1500 loss_train: 0.1311
Epoch: 2000 loss_train: 0.1234
Epoch: 2500 loss_train: 0.1167
Epoch: 3000 loss_train: 0.0529
Epoch: 3500 loss_train: 0.1240
Epoch: 4000 loss_train: 0.0772
Epoch: 4500 loss_train: 0.0521
Epoch: 5000 loss_train: 0.0526
Epoch: 5500 loss_train: 0.0902
 total time: 8.1317s
0.0
Epoch: 0000 loss_train: 1.9553
Epoch: 0500 loss_train: 0.3693
Epoch: 1000 loss_train: 0.2591
Epoch: 1500 loss_train: 0.1485
Epoch: 2000 loss_train: 0.1123
Epoch: 2500 loss_train: 0.1356
Epoch: 3000 loss_train: 0.0511
Epoch: 3500 loss_train: 0.1022
Epoch: 4000 loss_train: 0.0794
Epoch: 4500 loss_train: 0.0972
Epoch: 5000 loss_train: 0.0586
Epoch: 5500 loss_train: 0.0569
 total time: 8.3985s
0.22296759486198425
Epoch: 0000 loss_train: 1.9551
Epoch: 0500 loss_train: 0.3697
Epoch: 1000 loss_train: 0.2736
Epoch: 1500 loss_train: 0.1323
Epoch: 2000 loss_train: 0.1178
Epoch: 2500 loss_train: 0.1159
Epoch: 3000 loss_train: 0.0614
Epoc

Epoch: 0500 loss_train: 0.3630
Epoch: 1000 loss_train: 0.2648
Epoch: 1500 loss_train: 0.1602
Epoch: 2000 loss_train: 0.1316
Epoch: 2500 loss_train: 0.0989
Epoch: 3000 loss_train: 0.0613
Epoch: 3500 loss_train: 0.1160
Epoch: 4000 loss_train: 0.0698
Epoch: 4500 loss_train: 0.0667
Epoch: 5000 loss_train: 0.0575
Epoch: 5500 loss_train: 0.0727
 total time: 9.7186s
0.0
Epoch: 0000 loss_train: 1.9551
Epoch: 0500 loss_train: 0.3624
Epoch: 1000 loss_train: 0.2747
Epoch: 1500 loss_train: 0.1528
Epoch: 2000 loss_train: 0.1117
Epoch: 2500 loss_train: 0.1081
Epoch: 3000 loss_train: 0.0648
Epoch: 3500 loss_train: 0.1137
Epoch: 4000 loss_train: 0.0724
Epoch: 4500 loss_train: 0.0408
Epoch: 5000 loss_train: 0.0542
Epoch: 5500 loss_train: 0.0686
 total time: 11.0763s
1.7666984796524048
Epoch: 0000 loss_train: 1.9551
Epoch: 0500 loss_train: 0.3545
Epoch: 1000 loss_train: 0.2731
Epoch: 1500 loss_train: 0.1432
Epoch: 2000 loss_train: 0.1181
Epoch: 2500 loss_train: 0.1080
Epoch: 3000 loss_train: 0.0573
Epoc

Epoch: 0500 loss_train: 0.3685
Epoch: 1000 loss_train: 0.2601
Epoch: 1500 loss_train: 0.1635
Epoch: 2000 loss_train: 0.1138
Epoch: 2500 loss_train: 0.1144
Epoch: 3000 loss_train: 0.0642
Epoch: 3500 loss_train: 0.1165
Epoch: 4000 loss_train: 0.0618
Epoch: 4500 loss_train: 0.0643
Epoch: 5000 loss_train: 0.0678
Epoch: 5500 loss_train: 0.0612
 total time: 8.5105s
0.029665131121873856
Epoch: 0000 loss_train: 1.9559
Epoch: 0500 loss_train: 0.3730
Epoch: 1000 loss_train: 0.2466
Epoch: 1500 loss_train: 0.1752
Epoch: 2000 loss_train: 0.1128
Epoch: 2500 loss_train: 0.1019
Epoch: 3000 loss_train: 0.0652
Epoch: 3500 loss_train: 0.0961
Epoch: 4000 loss_train: 0.0673
Epoch: 4500 loss_train: 0.0756
Epoch: 5000 loss_train: 0.0626
Epoch: 5500 loss_train: 0.0794
 total time: 8.4705s
6.951605796813965
Epoch: 0000 loss_train: 1.9555
Epoch: 0500 loss_train: 0.3585
Epoch: 1000 loss_train: 0.2319
Epoch: 1500 loss_train: 0.1530
Epoch: 2000 loss_train: 0.0997
Epoch: 2500 loss_train: 0.1131
Epoch: 3000 loss_tra

Epoch: 0500 loss_train: 0.3933
Epoch: 1000 loss_train: 0.2518
Epoch: 1500 loss_train: 0.1650
Epoch: 2000 loss_train: 0.1298
Epoch: 2500 loss_train: 0.1246
Epoch: 3000 loss_train: 0.0596
Epoch: 3500 loss_train: 0.0870
Epoch: 4000 loss_train: 0.0669
Epoch: 4500 loss_train: 0.0699
Epoch: 5000 loss_train: 0.0676
Epoch: 5500 loss_train: 0.0623
 total time: 9.3757s
0.008787650614976883
Epoch: 0000 loss_train: 1.9546
Epoch: 0500 loss_train: 0.3703
Epoch: 1000 loss_train: 0.2630
Epoch: 1500 loss_train: 0.1551
Epoch: 2000 loss_train: 0.1194
Epoch: 2500 loss_train: 0.1097
Epoch: 3000 loss_train: 0.0677
Epoch: 3500 loss_train: 0.0693
Epoch: 4000 loss_train: 0.0681
Epoch: 4500 loss_train: 0.0488
Epoch: 5000 loss_train: 0.0361
Epoch: 5500 loss_train: 0.0347
 total time: 8.5370s
84.90425872802734
Epoch: 0000 loss_train: 1.9540
Epoch: 0500 loss_train: 0.4241
Epoch: 1000 loss_train: 0.2523
Epoch: 1500 loss_train: 0.1602
Epoch: 2000 loss_train: 0.1065
Epoch: 2500 loss_train: 0.1309
Epoch: 3000 loss_tra

...

Epoch: 1000 loss_train: 0.2382
Epoch: 1500 loss_train: 0.1320
Epoch: 2000 loss_train: 0.0988
Epoch: 2500 loss_train: 0.1452
Epoch: 3000 loss_train: 0.1013
Epoch: 3500 loss_train: 0.0632
Epoch: 4000 loss_train: 0.0577
Epoch: 4500 loss_train: 0.0502
Epoch: 5000 loss_train: 0.0772
Epoch: 5500 loss_train: 0.0686
 total time: 7.8854s
99.7408447265625
Epoch: 0000 loss_train: 1.9540
Epoch: 0500 loss_train: 0.4230
Epoch: 1000 loss_train: 0.2304
Epoch: 1500 loss_train: 0.1466
Epoch: 2000 loss_train: 0.1115
Epoch: 2500 loss_train: 0.1113
Epoch: 3000 loss_train: 0.1041
Epoch: 3500 loss_train: 0.0649
Epoch: 4000 loss_train: 0.0663
Epoch: 4500 loss_train: 0.0675
Epoch: 5000 loss_train: 0.0775
Epoch: 5500 loss_train: 0.0623
 total time: 7.8691s
0.49133366346359253
Epoch: 0000 loss_train: 1.9539
Epoch: 0500 loss_train: 0.4106
Epoch: 1000 loss_train: 0.2386
Epoch: 1500 loss_train: 0.1472
Epoch: 2000 loss_train: 0.1116
Epoch: 2500 loss_train: 0.1465
Epoch: 3000 loss_train: 0.1337
Epoch: 3500 loss_train

Epoch: 0500 loss_train: 0.4249
Epoch: 1000 loss_train: 0.2557
Epoch: 1500 loss_train: 0.1307
Epoch: 2000 loss_train: 0.1262
Epoch: 2500 loss_train: 0.1148
Epoch: 3000 loss_train: 0.0988
Epoch: 3500 loss_train: 0.0767
Epoch: 4000 loss_train: 0.0593
Epoch: 4500 loss_train: 0.0796
Epoch: 5000 loss_train: 0.0741
Epoch: 5500 loss_train: 0.0587
 total time: 7.8738s
33.113346099853516
Epoch: 0000 loss_train: 1.9525
Epoch: 0500 loss_train: 0.4223
Epoch: 1000 loss_train: 0.2506
Epoch: 1500 loss_train: 0.1352
Epoch: 2000 loss_train: 0.1169
Epoch: 2500 loss_train: 0.1274
Epoch: 3000 loss_train: 0.0848
Epoch: 3500 loss_train: 0.0747
Epoch: 4000 loss_train: 0.0625
Epoch: 4500 loss_train: 0.0611
Epoch: 5000 loss_train: 0.0682
Epoch: 5500 loss_train: 0.0634
 total time: 7.9006s
0.0005254556890577078
Epoch: 0000 loss_train: 1.9539
Epoch: 0500 loss_train: 0.4292
Epoch: 1000 loss_train: 0.2494
Epoch: 1500 loss_train: 0.1328
Epoch: 2000 loss_train: 0.1181
Epoch: 2500 loss_train: 0.1174
Epoch: 3000 loss_t

Epoch: 2000 loss_train: 0.0992
Epoch: 2500 loss_train: 0.1042
Epoch: 3000 loss_train: 0.1074
Epoch: 3500 loss_train: 0.0856
Epoch: 4000 loss_train: 0.0484
Epoch: 4500 loss_train: 0.0727
Epoch: 5000 loss_train: 0.0886
Epoch: 5500 loss_train: 0.0605
 total time: 8.0994s
3.8146897622937104e-06
Epoch: 0000 loss_train: 1.9527
Epoch: 0500 loss_train: 0.4336
Epoch: 1000 loss_train: 0.2427
Epoch: 1500 loss_train: 0.1305
Epoch: 2000 loss_train: 0.1075
Epoch: 2500 loss_train: 0.1085
Epoch: 3000 loss_train: 0.0859
Epoch: 3500 loss_train: 0.0747
Epoch: 4000 loss_train: 0.0543
Epoch: 4500 loss_train: 0.0766
Epoch: 5000 loss_train: 0.0786
Epoch: 5500 loss_train: 0.0539
 total time: 7.8544s
2.312633478140924e-05
Epoch: 0000 loss_train: 1.9528
Epoch: 0500 loss_train: 0.4381
Epoch: 1000 loss_train: 0.2545
Epoch: 1500 loss_train: 0.1244
Epoch: 2000 loss_train: 0.1108
Epoch: 2500 loss_train: 0.1019
Epoch: 3000 loss_train: 0.1020
Epoch: 3500 loss_train: 0.0879
Epoch: 4000 loss_train: 0.0583
Epoch: 4500 lo

Epoch: 4000 loss_train: 0.0544
Epoch: 4500 loss_train: 0.0686
Epoch: 5000 loss_train: 0.0594
Epoch: 5500 loss_train: 0.0663
 total time: 7.8624s
0.0
Epoch: 0000 loss_train: 1.9529
Epoch: 0500 loss_train: 0.4381
Epoch: 1000 loss_train: 0.2523
Epoch: 1500 loss_train: 0.1331
Epoch: 2000 loss_train: 0.1154
Epoch: 2500 loss_train: 0.1127
Epoch: 3000 loss_train: 0.0765
Epoch: 3500 loss_train: 0.0881
Epoch: 4000 loss_train: 0.0549
Epoch: 4500 loss_train: 0.0663
Epoch: 5000 loss_train: 0.0734
Epoch: 5500 loss_train: 0.0630
 total time: 7.8336s
0.0
Epoch: 0000 loss_train: 1.9528
Epoch: 0500 loss_train: 0.4385
Epoch: 1000 loss_train: 0.2544
Epoch: 1500 loss_train: 0.1302
Epoch: 2000 loss_train: 0.1138
Epoch: 2500 loss_train: 0.1118
Epoch: 3000 loss_train: 0.0818
Epoch: 3500 loss_train: 0.1003
Epoch: 4000 loss_train: 0.0613
Epoch: 4500 loss_train: 0.0595
Epoch: 5000 loss_train: 0.0761
Epoch: 5500 loss_train: 0.0772
 total time: 7.7953s
0.0
Epoch: 0000 loss_train: 1.9529
Epoch: 0500 loss_train: 0.

Epoch: 1000 loss_train: 0.2471
Epoch: 1500 loss_train: 0.1259
Epoch: 2000 loss_train: 0.1107
Epoch: 2500 loss_train: 0.1135
Epoch: 3000 loss_train: 0.1020
Epoch: 3500 loss_train: 0.0855
Epoch: 4000 loss_train: 0.0545
Epoch: 4500 loss_train: 0.0719
Epoch: 5000 loss_train: 0.0666
Epoch: 5500 loss_train: 0.0452
 total time: 7.7672s
0.0
Epoch: 0000 loss_train: 1.9528
Epoch: 0500 loss_train: 0.4397
Epoch: 1000 loss_train: 0.2588
Epoch: 1500 loss_train: 0.1353
Epoch: 2000 loss_train: 0.1100
Epoch: 2500 loss_train: 0.1083
Epoch: 3000 loss_train: 0.0893
Epoch: 3500 loss_train: 0.0968
Epoch: 4000 loss_train: 0.0516
Epoch: 4500 loss_train: 0.0791
Epoch: 5000 loss_train: 0.0839
Epoch: 5500 loss_train: 0.0656
 total time: 7.7789s
0.0
Epoch: 0000 loss_train: 1.9529
Epoch: 0500 loss_train: 0.4440
Epoch: 1000 loss_train: 0.2546
Epoch: 1500 loss_train: 0.1227
Epoch: 2000 loss_train: 0.1079
Epoch: 2500 loss_train: 0.1063
Epoch: 3000 loss_train: 0.0847
Epoch: 3500 loss_train: 0.0746
Epoch: 4000 loss_tra

In [35]:
with open(os.path.join(logsavepath,'crossVal_loss'), 'wb') as output:
    pickle.dump(predtest, output, pickle.HIGHEST_PROTOCOL)

In [36]:
predtest_label=np.argmax(predtest,axis=1)

In [37]:
res=pd.DataFrame({'sampleName':imgNamesAll,'true':progUnique[labelsAll.cpu().numpy()],'predicted':progUnique[predtest_label]})
res.to_csv(os.path.join(plotsavepath,'predictions.csv'))

In [38]:
#plot confusion
def plotCTcomp(labels,ctlist,savepath,savenamecluster,byCT,addname='',order=progInclude):
    res=np.zeros((order.size,order.size))
    for li in range(res.shape[0]):
        l=order[li]
        nl=np.sum(labels==l)
        ctlist_l=ctlist[labels==l]
        for ci in range(res.shape[1]):
            c=order[ci]
            res[li,ci]=np.sum(ctlist_l==c)
#             res[li,ci]=np.sum(ctlist_l==c)/nl
    if not byCT:
        addname+=''
        for li in range(res.shape[0]):
            l=order[li]
            nl=np.sum(labels==l)
            res[li]=res[li]/nl
    else:
        addname+='_normbyCT'
        for ci in range(res.shape[1]):
            c=order[ci]
            nc=np.sum(ctlist==c)
            res[:,ci]=res[:,ci]/nc
    
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(res,cmap='binary',vmin=0,vmax=1)
    fig.colorbar(im)
    ax.set_yticks(np.arange(order.size))
    ax.set_yticklabels(order)
    ax.set_xticks(np.arange(order.size))
    ax.set_xticklabels(order)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
    fig.tight_layout()
    plt.savefig(os.path.join(savepath,savenamecluster+addname+'.pdf'))
    plt.close()
    
plotCTcomp(progUnique[labelsAll.cpu().numpy()],progUnique[predtest_label],plotsavepath,'confusion'+str(testepoch),False)